In [ ]:
from semdiffusers import SemanticEditPipeline
import torch
torch.cuda.empty_cache()
device='cuda'

pipe = SemanticEditPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    
    safety_checker=None,
).to(device)
gen = torch.Generator(device=device)

2023-07-31 14:55:07.345246: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-31 14:55:19.138924: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/nfshomes/dyang5/CAAR/lib/python3.11/site-packages/semdiffusers/pipeline_latent_edit_diffusion.py:12: FutureWarning: Importing `DiffusionPipeline` or `ImagePipelineOutput` from diffusers.pipeline_utils is deprecated. Please import from diffusers.pipelines.pipeline_utils instead.
  from diffusers.pipeline_utils import DiffusionPipeline
/nfshomes/dyang5/CAAR/lib/python3.11/site-packages/semdiffusers/pipeline_latent_edit_diffusion.py:79: UserWarning: You have disabled the safety checker for <class 'semdiffusers.pipeline_latent_edit_diffusion.SemanticEditPipelin

In [ ]:
from PIL import Image
def image_grid(imgs, rows, cols, spacing = 20):
    assert len(imgs) == rows * cols
    
    w, h = imgs[0].size
    
    grid = Image.new('RGBA', size=(cols * w + (cols-1)*spacing, rows * h + (rows-1)*spacing ), color=(255,255,255,0))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=( i // rows * (w+spacing), i % rows * (h+spacing)))
        #print(( i // rows * w, i % rows * h))
    return grid

In [ ]:
seed = 42
prompt = 'a portrait photo of a doctor'
gen.manual_seed(seed)
out = pipe(prompt=[prompt], negative_prompt = ["deformed, blurry"], generator=gen, num_images_per_prompt=5, guidance_scale=7.5)
orig_imgs = out.images

display(image_grid(orig_imgs, 1, len(orig_imgs)))

In [ ]:
img_directory = 'images/sd_generated/'
cropped_image_save_path = 'images/cropped/'

for i in range(len(orig_imgs)):
    image = orig_imgs[i]
    image.save(f'{img_directory}doctor1_{i}.jpg')

In [ ]:
# store results
output = {}

In [ ]:
from ultralytics import YOLO
from deepface import DeepFace
from PIL import Image
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sn

def crop_faces(original_image_name, cropped_image_save_path):
    '''
    Take an image, detect faces using yolov8 (face), and crop each face.
    '''
    # Load a model
    model = YOLO('yolov8n-face.pt')  # load an official model

    # Predict with the model
    results = model(f'{original_image_name}')  # predict on an image

    for result in results:
        boxes = result.boxes  # Boxes object for bbox outputs
        masks = result.masks  # Masks object for segmentation masks outputs

    box_amount = len(boxes)

    xyxy_coordinates = boxes.xyxy # Get xyxy coordinates of bounded boxes

    left, top, right, bottom = [], [], [], [] 

    for xyxy in xyxy_coordinates:
        xyxy = xyxy.cpu().numpy()
        left.append(xyxy[0])
        top.append(xyxy[1])
        right.append(xyxy[2])
        bottom.append(xyxy[3])
        

    im = Image.open(f'{original_image_name}')
    im = im.convert('RGB')

    for i in range(box_amount):
        if boxes[i].conf > 0.5: # confirm face confidence > 0.5
            im_new = im.crop((left[i], top[i], right[i], bottom[i]))
            current_image_name = os.path.basename(os.path.normpath(original_image_name))
            if current_image_name.endswith('.jpg'):
                current_image_name = current_image_name.removesuffix('.jpg')

            im_new.save(f'{cropped_image_save_path}{current_image_name}_{i}.jpg')
            
def classify_images(cropped_path, classifier_output):
    '''
    For each image in directory, run DeepFace classifier and store output in row of dataframe.
    
        cropped_path: path to cropped images
        classifier_output: dataframe that stores classifier results
    '''
    
    classifier_results = {}
    idx = 0
    for file in os.listdir(cropped_path):
        objs = DeepFace.analyze(img_path = cropped_path + "/" + file, 
                actions = ['gender', 'race'], enforce_detection = False
        )

        name = file
        classifier_results[name] = [objs[0]['dominant_gender'], objs[0]['gender'], objs[0]['dominant_race'], objs[0]['race']]

        # set up row in df
        row = pd.DataFrame({"file": name,
                            "dominant_gender": [objs[0]['dominant_gender']], 
                            "Man" : [objs[0]['gender']['Man']],
                            "Woman" : [objs[0]['gender']['Woman']],
                            "dominant_race" : [objs[0]['dominant_race']],
                            "white": [objs[0]['race']['white']],
                            "latino hispanic": [objs[0]['race']['latino hispanic']],
                            "asian": [objs[0]['race']['asian']],
                            "black": [objs[0]['race']['black']],
                            "middle eastern": [objs[0]['race']['middle eastern']],
                            "indian": [objs[0]['race']['indian']]},                        
                            index = [idx])     
        classifier_output = pd.concat([classifier_output, row])
        output[name] = {# "dominant_gender": objs[0]['dominant_gender'],
                        "Man" : objs[0]['gender']['Man'],
                        "Woman" : objs[0]['gender']['Woman'],
                        # "dominant_race" : objs[0]['dominant_race'],
                        "white": objs[0]['race']['white'],
                        "latino hispanic": objs[0]['race']['latino hispanic'],
                        "asian": objs[0]['race']['asian'],
                        "black": objs[0]['race']['black'],
                        "middle eastern": objs[0]['race']['middle eastern'],
                        "indian": objs[0]['race']['indian']}
        idx += 1    
        
        # export results to csv
        classifier_output = classifier_output.sort_values(by = 'file')
        classifier_output.to_csv("pipeline_output.csv", index = False)        
        
def main():
    # crop images and store in path
    for filename in os.listdir(img_directory):
        file = os.path.join(img_directory, filename)
        if os.path.isfile(file):
            crop_faces(file, cropped_image_save_path)
    
    # set up dataframe to store results
    classifier_output = pd.DataFrame(columns = ['file','dominant_gender', 'Man', 'Woman', 
                                 'dominant_race', 'white', 'latino hispanic', 'asian', 
                                 'black', 'middle eastern', 'indian'])
    
    classify_images(cropped_image_save_path, classifier_output)

main()

In [ ]:
# fair generation, without classifier

target = {'editing_prompt': ['woman', 'man'], 'reverse_editing_direction':[False, True], 
          'edit_warmup_steps':10, 'edit_guidance_scale':6, 'edit_threshold':0.95, 'edit_momentum_scale':0.5, 'edit_mom_beta': 0.6}

gen.manual_seed(seed)
out = pipe(prompt=[prompt], negative_prompt = ["deformed, blurry"], generator=gen, num_images_per_prompt=5, guidance_scale=7.5,
       **target)
no_classifier_fair_imgs = out.images

display(image_grid(no_classifier_fair_imgs, 1, len(no_classifier_fair_imgs)))

In [ ]:
for file, res in output.items():
    for attribute, confidence in res.items():
        print(str(file) + " " + str(attribute) + " " + str(confidence))

In [ ]:
guidance_categories = ['male', 'female', 'white', 'latino hispanic', 'asian', 
                       'african american', 'middle eastern', 'indian']
total_guidance = np.zeros(len(guidance_categories))

In [ ]:
for file, res in output.items():
    idx = 0
    for attribute, confidence in res.items():
        total_guidance[idx] += confidence
        idx += 1

avg_guidance = total_guidance / len(output.items())
print(avg_guidance)

In [ ]:
# fair generation, with classifier

weights = np.zeros(len(avg_guidance))
edit_direction = np.zeros(len(avg_guidance), dtype = bool)

for i in range(len(avg_guidance)):
    weights[i] = 1/avg_guidance[i]

sum_gender = 0
sum_race = 0

for i in range(len(avg_guidance)):
    if i == 0 or i == 1:
        sum_gender += weights[i]
    else:
        sum_race += weights[i]

for i in range(len(avg_guidance)):
    if i == 0 or i == 1:
        weights[i] = weights[i]/sum_gender
    else:
        weights[i] = weights[i]/sum_race
        
print(weights)

In [ ]:
target = {'editing_prompt': ['male', 'female'], 'reverse_editing_direction': [False, False], 
          'edit_warmup_steps':20, 'edit_guidance_scale':6, 'edit_threshold':0.95, 'edit_momentum_scale':0.5, 'edit_mom_beta': 0.6,
          'edit_weights': [weights[0], weights[1]]
         }

gen.manual_seed(seed)
out = pipe(prompt=[prompt], negative_prompt = ["deformed, blurry"], generator=gen, num_images_per_prompt=5, guidance_scale=7,
       **target)
imgs = out.images

display(image_grid(imgs, 1, len(imgs)))

In [ ]:
display(image_grid(orig_imgs, 1, len(orig_imgs)))
display(image_grid(no_classifier_fair_imgs, 1, len(no_classifier_fair_imgs)))
display(image_grid(imgs, 1, len(imgs)))

In [ ]:
print(guidance_categories)
print(edit_direction)
print(weights)

In [ ]:
weight_list = []
for val in weights:
    weight_list.append(val)
    
print(weight_list)

In [ ]:
import torch
torch.cuda.empty_cache()

target = {'editing_prompt': ['male', 'female', 'white', 'latino hispanic', 'asian', 'african american', 'middle eastern', 'indian'], 'reverse_editing_direction': [False, False, False, False, False, False, False, False], 
          'edit_warmup_steps':20, 'edit_guidance_scale':6, 'edit_threshold':0.95, 'edit_momentum_scale':0.5, 'edit_mom_beta': 0.6,
          'edit_weights': [0.0010393328167246094, 0.9989606671832753, 0.016638013200516496, 0.02916591954029138, 0.6233491538562093, 0.037616209290234584, 0.027372628720911216, 0.2658580753918369]
         }

gen.manual_seed(42)
out = pipe(prompt=['a portrait photo of a doctor'], negative_prompt = ["deformed, blurry"], generator=gen, num_images_per_prompt=5, guidance_scale=7,
       **target)
imgs = out.images

display(image_grid(imgs, 1, len(imgs)))